### Takes images from Generating_images.ipynb

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
#import warnings
#import csv
import gc
from time import time
import logging
#from scipy.io import loadmat
#from select_win import select_win,select_windows
import load_data as ldfile
#import create_model as cmodel
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.device('/device:GPU:0')

#from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
#from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
from ecg_detectors.ecgdetectors import Detectors, MWA, panPeakDetect, searchBack

np.random.seed(354)
#sns.set()
#warnings.filterwarnings('ignore')
#logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.random.set_seed(1342) # Set seed for reproducibility

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")


Default GPU Device:/device:GPU:0


### Dataset Creation

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10898739640230800900
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12671107054397672548
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12663305944450672164
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10561793600
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10773054088001040570
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5"
]


In [2]:
# generate the datasets for step 8:
train_d = np.load('train_v1.npy')
val_d = np.load('val_v1.npy')
train_lab = np.load('train_lab_v1.npy')
val_lab = np.load('val_lab_v1.npy')

train_set_conv,valid_set_conv = ldfile.load_tf(train_d,train_lab,val_d,val_lab)


print ("train_im, train_lab types: ", type(train_d), type(train_lab))
#### check the shape of the data
print ("shape of images and labels array: ", train_d.shape, train_lab.shape) 
print ("shape of images and labels array ; test: ", val_d.shape, val_lab.shape)


train_dataset type:  <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
val_dataset type:  <class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
train_im, train_lab types:  <class 'numpy.ndarray'> <class 'numpy.ndarray'>
shape of images and labels array:  (26828, 400, 200, 3) (26828, 9)
shape of images and labels array ; test:  (3532, 400, 200, 3) (3532, 9)


In [3]:
del train_d, val_d, train_lab, val_lab
gc.collect()

22

In [24]:
train_d = [26828, 12, 2500, 1]
val_d = [3532, 12, 2500, 1]
train_lab = [26828, 9]
val_lab = [3532, 9]

In [3]:
with tf.device("/GPU:0"):
    t1 = time()
    input_directory = '/dataset/PhysioNet2020/Training_WFDB' # Dataset on beastie 
    train_data,train_labels_bin,val_data,val_labels_bin = ldfile.load_data2D_bin(input_directory) 
    train_set_conv,valid_set_conv = ldfile.load_tf(train_data,train_labels_bin,val_data,val_labels_bin)
    t2 = time()
    print('Loading data in {} seconds.'.format(round(t2-t1)))


total number of files:  6877
Number of files for trainig:  5501
Number of files for validation:  687
Number of files for testing:  687


/home/siddiqmk/.local/lib/python3.8/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


500 files were loaded for training!
1000 files were loaded for training!
1500 files were loaded for training!
2000 files were loaded for training!
2500 files were loaded for training!
3000 files were loaded for training!
3500 files were loaded for training!
4000 files were loaded for training!
4500 files were loaded for training!
5000 files were loaded for training!
5500 files were loaded for training!
499 files were loaded for validation!
Labels:
0. 164884008
1. 164889003
2. 164909002
3. 164931005
4. 270492004
5. 284470004
6. 426783006
7. 429622005
8. 59118001

number of test signals =  689

train_data type: <class 'list'>
train_labels  type: <class 'list'>
val_data  type: <class 'list'>
val_labels  type: <class 'list'>

train_data length 26828
train_labels length 26828
val_data length: 3532
val_labels length: 3532

train_data type: <class 'numpy.ndarray'>
train_labels_bin  type: <class 'numpy.ndarray'>
val_data  type: <class 'numpy.ndarray'>
val_labels_bin  type: <class 'numpy.ndarra

In [4]:
del train_data,train_labels_bin,val_data,val_labels_bin
gc.collect()

22

# RESNET 50

In [25]:
#### Necessary Imports for Neural Net 

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, ZeroPadding2D,\
     Flatten, BatchNormalization, AveragePooling2D, Dense, Activation, Add 
from tensorflow.keras.models import Model
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [26]:
def res_identity(x, filters): 
    ''' renet block where dimension doesnot change.
    The skip connection is just simple identity conncection
    we will have 3 blocks and then input will be added
    '''
    x_skip = x # this will be used for addition with the residual block 
    f1, f2 = filters

    #first block 
    x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    #second block # bottleneck (but size kept same with padding)
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    # third block activation used after adding the input
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    # x = Activation(activations.relu)(x)

    # add the input 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [27]:
def res_conv(x, s, filters):
    '''
    here the input size changes, when it goes via conv blocks
    so the skip connection uses a projection (conv layer) matrix
    ''' 
    x_skip = x
    f1, f2 = filters

    # first block
    x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x)
    # when s = 2 then it is like downsizing the feature map
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    # second block
    x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)

    #third block
    x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)

    # shortcut 
    x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_regularizer=l2(0.001))(x_skip)
    x_skip = BatchNormalization()(x_skip)

    # add 
    x = Add()([x, x_skip])
    x = Activation(activations.relu)(x)

    return x

In [28]:
### Combine the above functions to build 50 layers resnet. 
def resnet50():
    
    #input_im = Input(shape=(train_d.shape[1], train_d.shape[2], train_d.shape[3])) # cifar 10 images size
    input_im = Input(shape=(train_d[1], train_d[2], train_d[3])) # train_d
    x = ZeroPadding2D(padding=(3, 3))(input_im)

    # 1st stage
    # here we perform maxpooling, see the figure above

    x = Conv2D(64, kernel_size=(7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation(activations.relu)(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #2nd stage 
    # frm here on only conv block and identity block, no pooling

    x = res_conv(x, s=1, filters=(64, 256))
    x = res_identity(x, filters=(64, 256))
    x = res_identity(x, filters=(64, 256))

    # 3rd stage

    x = res_conv(x, s=2, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))
    x = res_identity(x, filters=(128, 512))

    # 4th stage

    x = res_conv(x, s=2, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))
    x = res_identity(x, filters=(256, 1024))

    # 5th stage

    x = res_conv(x, s=2, filters=(512, 2048))
    x = res_identity(x, filters=(512, 2048))
    x = res_identity(x, filters=(512, 2048))

    # ends with average pooling and dense connection

    x = AveragePooling2D((2, 2), padding='same')(x)

    x = Flatten()(x)
    #x = Dense(len(class_types), activation='softmax', kernel_initializer='he_normal')(x) #multi-class
    x = Dense(9, activation='softmax', kernel_initializer='he_normal', name ='visualized_layer')(x) #multi-class
    # define the model 

    with tf.device("/GPU:2"):
        model = Model(inputs=input_im, outputs=x, name='Resnet50')

    return model

In [29]:
### Define some Callbacks
def lrdecay(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    #print('Learning rate: ', lr)
    return lr
  # if epoch < 40:
  #   return 0.01
  # else:
  #   return 0.01 * np.math.exp(0.03 * (40 - epoch))
lrdecay = tf.keras.callbacks.LearningRateScheduler(lrdecay) # learning rate decay  


def earlystop(mode):
    if mode=='acc':
        estop = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=15, mode='max')
    elif mode=='loss':
        estop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, mode='min')
    return estop    

In [30]:
resnet50_model = resnet50()

In [31]:
resnet50_model.summary()

Model: "Resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 12, 2500, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 18, 2506, 1)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_106 (Conv2D)             (None, 6, 1250, 64)  3200        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
batch_normalization_106 (BatchN (None, 6, 1250, 64)  256         conv2d_106[0][0]                 
___________________________________________________________________________________________

In [32]:
resnet50_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), 
                   metrics=['acc'])

In [44]:
batch_size=1000 # test with 64, 128, 256
with tf.device("/GPU:0"):
    resnet_train = resnet50_model.fit(train_set_conv, 
                                  epochs=5, 
                                  #verbose = 2,
                                  #steps_per_epoch=train_im.shape[0]/batch_size,
                                  #steps_per_epoch=train_d.shape[0]/batch_size,
                                  steps_per_epoch=train_d[0]/batch_size, #for train_d
                                  #validation_steps=valid_im.shape[0]/batch_size, 
                                  #validation_steps=val_d.shape[0]/batch_size, 
                                  validation_steps=val_d[0]/batch_size,
                                  validation_data=valid_set_conv, 
                                  callbacks=[lrdecay])

Epoch 1/5
27/26 [==============================] - 6s 210ms/step - loss: 432.4942 - acc: 0.2002 - val_loss: 347.6226 - val_acc: 0.1992
Epoch 2/5
27/26 [==============================] - 6s 209ms/step - loss: 619.1556 - acc: 0.1574 - val_loss: 396.8588 - val_acc: 0.1875
Epoch 3/5
27/26 [==============================] - 6s 209ms/step - loss: 785.7731 - acc: 0.1348 - val_loss: 412.6528 - val_acc: 0.1602
Epoch 4/5
27/26 [==============================] - 6s 210ms/step - loss: 668.4009 - acc: 0.1719 - val_loss: 771.7839 - val_acc: 0.0742
Epoch 5/5
27/26 [==============================] - 6s 211ms/step - loss: 584.5294 - acc: 0.1591 - val_loss: 468.6450 - val_acc: 0.1328


In [50]:
### Plot train and validation curves
loss = resnet_train.history['loss']
v_loss = resnet_train.history['val_loss']

acc = resnet_train.history['acc']
v_acc = resnet_train.history['val_acc']

epochs = range(len(loss))

fig = plt.figure(figsize=(9, 5))
plt.subplot(1, 2, 1)
plt.yscale('log')
plt.plot(epochs, loss, linestyle='--', linewidth=3, color='orange', alpha=0.7, label='Train Loss')
plt.plot(epochs, v_loss, linestyle='-.', linewidth=2, color='lime', alpha=0.8, label='Valid Loss')
plt.ylim(0.01, 10000)
plt.xlabel('Epochs', fontsize=11)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=12)
plt.subplot(1, 2, 2)
plt.plot(epochs, acc, linestyle='--', linewidth=3, color='orange', alpha=0.7, label='Train Acc')
plt.plot(epochs, v_acc, linestyle='-.', linewidth=2, color='lime', alpha=0.8, label='Valid Acc') 
plt.xlabel('Epochs', fontsize=11)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig('train_acc.png', dpi=250)
plt.show()

In [46]:
#### Plot the Confusion Matrix

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def conf_matrix(predictions): 
    ''' Plots conf. matrix and classification report '''
    cm=confusion_matrix(test_lab, np.argmax(np.round(predictions), axis=1))
    print("Classification Report:\n")
    cr=classification_report(test_lab,
                                np.argmax(np.round(predictions), axis=1), 
                                target_names=[class_types[i] for i in range(len(class_types))])
    print(cr)
    plt.figure(figsize=(12,12))
    sns_hmp = sns.heatmap(cm, annot=True, xticklabels = [class_types[i] for i in range(len(class_types))], 
                yticklabels = [class_types[i] for i in range(len(class_types))], fmt="d")
    fig = sns_hmp.get_figure()
    fig.savefig('heatmap.png', dpi=250)

In [39]:
pred_class_resnet50 = resnet50_model.predict(val_im)

conf_matrix(pred_class_resnet50)

NameError: name 'val_im' is not defined

In [42]:
### Resutls on Test Data; 
## Check the performance on the test data 
test_result = resnet50_model.evaluate(test_im, test_lab_categorical, verbose=0)

print ("ResNet50 loss: ", test_result[0])
print ("ResNet50 accuracy: ", test_result[1])

ResNet50 loss:  1.1827166080474854
ResNet50 accuracy:  0.6662999987602234


In [25]:
valid_im[0].shape

(32, 32, 3)